# **Why large language models break down for computation**
normaly LLM takes process over the tokens how the model is tokenizing string is where it all comes down at the end so for mathmetical operation like:
24 + 433
so different models might create different tokens for this string  
eg 1 : '24' ,  ' +'   ,' 4'   ,' 33'
also for stings also 'hello how are you tokenization'
tokena 'hello', ' how',' are',' you',' token','ization'



****remember the space before each token it might differe model to model****



another example :
Egg. -------->  'E' ,'gg'<br>
I eat Egg.  ---> 'I',' eat','Egg.'<br>
see how same Egg. getting broken down to two when used with 'Space'

https://tiktokenizer.vercel.app/



In [1]:
# UTF-8
list("hello GPT".encode("utf-8"))

[104, 101, 108, 108, 111, 32, 71, 80, 84]

**LLM models can be built tokenization free by directly feeding byte streeem though the it may not be fesiable becuase now transformer has to consider large context length**


# for to use unicode with LLM we use byte pair Encoding
it applies like most repeated pair will create new token,and thus our vocab size will increase but context length decrease

'aabbaabaac'

most repeated char 'aa' -> Z
now we have string as 'ZbbZbZc' vocab({Z,b,c,a}, 4)

'bb' -> Y
now we have string as 'ZYZbZc' vocab({Z,b,c,Y,a}, 5)





In [2]:
text='''although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.'''
list_uni=list(text.encode("utf-8"))

In [3]:
# now we find the most common repeated pair in byte list
def get_count(_list):
    counts={}
    for pair in zip(_list,_list[1:]):
        counts[pair]=counts.get(pair,0)+1 
    return counts


In [4]:
count_dict=get_count(list_uni)
# count_dict=sorted(((v,k) for k,v in count_dict.items()),reverse=True)
chr(101),chr(32),chr(226), chr(128)

('e', ' ', 'â', '\x80')

In [5]:
def merge(_list,pair,idx):
    new_ids=[]
    i=0
    while i<len(_list):
        if i<len(_list)-1 and _list[i]==pair[0] and _list[i+1]==pair[1]:
            new_ids.append(idx)
            i+=2
        else:
            new_ids.append(_list[i])
            i+=1
            
    return new_ids

In [6]:
top_pair=max(count_dict,key=count_dict.get)
new_tokens=merge(list_uni,top_pair,256) # replace top repeating pair with new 256 token
len(new_tokens),len(list_uni)

(362, 372)

In [7]:
text='''We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a fileThe most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.UTF-8In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:UTF-8 (binary)	Code point (binary)	Range0xxxxxxx	xxxxxxx	U+0000–U+007F110xxxxx 10yyyyyy	xxxxxyyyyyy	U+0080–U+07FF1110xxxx 10yyyyyy 10zzzzzz	xxxxyyyyyyzzzzzz	U+0800–U+FFFF11110xxx 10yyyyyy 10zzzzzz 10wwwwww	xxxyyyyyyzzzzzzwwwwww	U+10000–U+10FFFFA handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.UTF-16The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:UTF-16 (binary)	Code point (binary)	Rangexxxxxxxxxxxxxxxx	xxxxxxxxxxxxxxxx	U+0000–U+FFFF110110xxxxxxxxxx 110111yyyyyyyyyy	xxxxxxxxxxyyyyyyyyyy + 0x10000	U+10000–U+10FFFFA more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.'''
list_uni=list(text.encode("utf-8"))
len(list_uni)

4294

In [8]:
def get_count(_list):
    counts={}
    for pair in zip(_list,_list[1:]):
        counts[pair]=counts.get(pair,0)+1 
    return counts

def merge(_list,pair,idx):
    new_ids=[]
    i=0
    while i<len(_list):
        if i<len(_list)-1 and _list[i]==pair[0] and _list[i+1]==pair[1]:
            new_ids.append(idx)
            i+=2
        else:
            new_ids.append(_list[i])
            i+=1
            
    return new_ids

###############################################################
vocab_size=276
new_tokens=276-256
list_uni_copy=list(list_uni)

merges={}
for i in range(new_tokens):
    uni_count=get_count(list_uni_copy)
    top_uni=max(uni_count,key=uni_count.get)
    print(f"merging {top_uni} to {256+i}")
    list_uni_copy=merge(list_uni_copy,top_uni,256+i)
    merges[top_uni]=256+i
    

merging (101, 32) to 256
merging (105, 110) to 257
merging (120, 120) to 258
merging (115, 32) to 259
merging (116, 104) to 260
merging (99, 111) to 261
merging (116, 32) to 262
merging (111, 114) to 263
merging (44, 32) to 264
merging (101, 114) to 265
merging (121, 121) to 266
merging (101, 110) to 267
merging (261, 100) to 268
merging (100, 32) to 269
merging (226, 128) to 270
merging (257, 103) to 271
merging (97, 110) to 272
merging (258, 258) to 273
merging (121, 32) to 274
merging (116, 101) to 275


In [9]:
len(list_uni_copy),len(list_uni),((len(list_uni)-len(list_uni_copy)) / len(list_uni))

(3333, 4294, 0.22380065207265953)

Tokenizers has diferent dataset then the LLM models and these are trained on vocublary using "**byte pair encoding**" (**BPE**)




# utf-8 must follow following format for byte representation
# 
# 0XXXXXXX or 110XXXXX or 1110XXXX or 11110XXX

128 doest follow the utf standard because 10000000




In [10]:
vocab={id:bytes([id]) for id in range(256)}

for (p0,p1),k in merges.items():
    vocab[k]=vocab[p0]+vocab[p1]
    
def decode(ids):
    tokens=b"".join(vocab[id] for id in ids)
    text=tokens.decode("utf-8",errors='replace')
    return text


# 97  ---->  bytes(97)  ---->  'a'

# 256  ---->  bytes(256)  ---->  'e  '

# 265  ---->  bytes(265)  ---->  'er'

In [11]:
def encode(text):
    tokens=list(text.encode('utf-8'))
    while len(tokens)>=2:
        token_dict=get_count(tokens)
        pair= min(token_dict,key=lambda p:merges.get(p,float('inf'))) 
        #float('inf') is used to make sure we don't include those pair which are not in our merge
        if pair not in merges:
            break #nothing to merge  
        idx=merges[pair]
        tokens=merge(tokens,pair,idx)
    return tokens

In [12]:
print(decode(encode('hello world!')))

hello world!


In [13]:
decode([263])

'or'

# Forced spilt using regex pattern(GPT series)

In [28]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

# gpt4pat=r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
# i for case insenstive

# " ?\p{L}" --->               ' ?'optional space followed by p{L} which means letter of anykind+ for following e.g it is enough
# " ?\p{N}"--->                any neumeric character
# " ?[^\s\p{L}\p{N}]+" --->    group special char
# "\s+(?!\S)"         --->     include alll white spaces in seq but not last one '(?!\S)'\
#  "       dog" -------------->      '       ', ' dog'

re.findall(gpt2pat,"hellow i've dog!???        dog123    ")

['hellow', ' i', "'ve", ' dog', '!???', '       ', ' dog', '123', '    ']

In [35]:
# !pip install tiktoken
import tiktoken

# gpt2
enc=tiktoken.get_encoding("gpt2")
print(enc.encode("     what is the status"))

enc=tiktoken.get_encoding("cl100k_base")
print(enc.encode("     what is the status3456"))

[220, 220, 220, 220, 644, 318, 262, 3722]
[257, 1148, 374, 279, 2704, 12901, 21]


# SentencePiece
----------------------------------------
The big difference between tiktoken vs sentencepiece is sentencepiece runs BPE on the Unicode code points directly .it then has an option character_coverage for what to do with very very rare codepoints that apperar very few times, and it either map them to <UNK> or if byte_fallback is turned on it encodes them with utf-8 and then encoedes the raw bytes instead **

In [51]:
import sentencepiece as spm

In [54]:
import os 
options=dict(
    input="""The phrase "problem solved itself" is often used figuratively to describe situations where a problem appears to have been resolved without direct intervention or effort from individuals involved. However, in reality, there's usually some underlying mechanism or action that led to the resolution, even if it's not immediately obvious.Here are a few ways a problem might appear to solve itself: Natural Processes: Sometimes, problems may seem to resolve on their own due to natural processes or phenomena. For example, if there's a conflict between two parties and one party unexpectedly decides to withdraw, the problem may appear to have resolved itself, but it's actually due to a decision made by one of the parties. Time: Some problems may resolve over time without direct intervention. This could be due to changing circumstances, attitudes, or perspectives. For instance, a disagreement between coworkers might gradually dissipate as they spend more time working together and gaining a better understanding of each other's viewpoints. Random Chance: In some cases, random chance or luck might seemingly resolve a problem. For instance, if there's a technical issue with a computer system that suddenly corrects itself without any intervention, it might appear as though the problem solved itself, when in reality, it was just a temporary glitch. Unintended Consequences: Occasionally, actions taken to address a problem can inadvertently lead to its resolution through unintended consequences. For example, implementing a new policy to address one issue might inadvertently resolve another unrelated problem. It's important to recognize that while a problem may seem to have solved itself, there are usually underlying factors at play, even if they're not immediately apparent. In many cases, understanding these underlying factors can help prevent similar problems from arising in the future or provide insights into more effective problem-solving strategies. User how does problem solved it self ChatGPT In some cases, a problem may appear to solve itself due to various factors. Here are a few ways this can happen:Self-Correction: Sometimes, systems have built-in mechanisms that automatically correct issues. For instance, in a biological system, the body's immune system can often resolve infections without external intervention. Similarly, in a technical system, error-detection and error-correction mechanisms can sometimes fix problems automatically.External Factors: Changes in external circumstances can sometimes resolve a problem without any direct action. For example, if a business is struggling due to economic downturn, an improvement in the economy might lead to increased sales and revenue, seemingly resolving the problem without any specific action taken by the business.Spontaneous Resolution: Some problems may simply resolve on their own over time due to natural fluctuations or random chance. For example, conflicts between individuals or groups might gradually dissipate as emotions cool down or as people's priorities shift.Learning and Adaptation: Individuals or systems involved in a problem may learn from their experiences and adapt their behavior accordingly, leading to a resolution over time. For instance, if a software program encounters a bug that causes it to crash, developers might observe and analyze the issue, then release a patch that fixes the bug, seemingly resolving the problem without direct intervention from users.Complex Systems Dynamics: In complex systems, problems can sometimes be self-organizing or self-regulating. Emergent properties or feedback loops within the system can lead to problem-solving behaviors without any centralized control. For example, in ecological systems, predator-prey dynamics can naturally regulate populations without external intervention.While it may seem like a problem has solved itself, it's often the result of underlying processes or changes occurring within the system. Recognizing these mechanisms can help us understand how problems are resolved and how to facilitate more effective problem-solving in the future.""",
    input_format="text",
#     model_prefix="tok100", #outputfilename
    model_type="bpe",
    vocab_size=100,
    normalization_rule_name="identity",
    remove_extra_whitespaces=False,
    input_sentence_size=1000000,
    shuffle_input_sentence=True,
    character_coverage=0.9995,
    byte_fallback=True,
    unk_id=0,
    bos_id=1,
    eos_id=2,
    pad_id=-1,
    num_threads=os.cpu_count()
)
spm.SentencePieceTrainer.Train(**options)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: The phrase "problem solved itself" is often used figuratively to describe situations where a problem appears to have been resolved without direct intervention or effort from individuals involved. However
  input:  in reality
  input:  there's usually some underlying mechanism or action that led to the resolution
  input:  even if it's not immediately obvious.Here are a few ways a problem might appear to solve itself: Natural Processes: Sometimes
  input:  problems may seem to resolve on their own due to natural processes or phenomena. For example
  input:  if there's a conflict between two parties and one party unexpectedly decides to withdraw
  input:  the problem may appear to have resolved itself
  input:  but it's actually due to a decision made by one of the parties. Time: Some problems may resolve over time without direct intervention. This could be due to changing circumstances
  input:  attit

RuntimeError: Internal: src/trainer_interface.cc(335) [(output_model_proto_ != nullptr && trainer_spec_.model_prefix().empty()) || (output_model_proto_ == nullptr && !trainer_spec_.model_prefix().empty())] ModelProto and trainer_spec.model_prefix() must be exclusive.

In [ ]:
sp=spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab=[[sp.id_to_pieace(idx),idx] for idx in range(sp.get_piece_size())]

**why we can not create vocab_size to "inf or 1 million"?**
- with increasing size of vocab there will be embeddings for each token also which must be trained , so in case of 1 million vocab_size these token will become rare to encounter in train set because so these embeddings might remain under trained,also if it is possible we train a very very huge data ,now we can cover more context with new squished tokens but model may not get time for each knowledgefull token to consider while predicting next token


- also model may behave differently when they see 'hello how ' space at the end ,because you have added the ' ' which will be part of next token like ' are',and model has not seen more ' ' wth any text following also those token which are not part of token dataset '.DefaultCellsty' called undatable token